# Geocoding Addresses

The vast majority of address data comes in the form of a string which is generally not directly usable by machine learning models. Therefore one important task data scientists face when handling geospatial data is the conversion of addresses to a numerical form which can then be used by machine learning models. This conversion is called <a href="https://en.wikipedia.org/wiki/Address_geocoding" target="_blank">geocoding</a>, and converts an address in string format to its geographical coordinates. For example, the <a href="https://en.wikipedia.org/wiki/White_House" target="_blank">White House</a>'s address at 1600 Pennsylvania Avenue NW geocodes to 38°53′51.7189″N, 77°02′11.5913″W. 

The <a href="https://geopy.readthedocs.io/en/stable/#nominatim" target="_blank">geopy</a> library is one of the more well known libraries for geocoding addresses in Python, although strictly speaking, geopy is simply a client which accesses the geocoding web services of third party providers. When using geopy, a geocoding web service such as <a href="https://developers.google.com/maps/documentation/geocoding/overview" target="_blank">Google Maps</a> or <a href="https://nominatim.org" target="_blank">Nominatim</a> must therefore be selected. Unfortunately, not all geocoding web services are provided for free, or have various restrictions such as daily limits for non-paying users. For this blog post, we will use <a href="https://nominatim.org" target="_blank">Nominatim</a>'s geocoding services which sets a strict limit of not more than 1 request per second.

Accessing Nominatim's geocoding web service using geopy is not too difficult, simply call `Nominatim()` from `geopy.geocoders`, specifying explicitly `user_agent` in the arguments. If `user_agent` is not explicitly specified, 403 or 429 HTTP errors might result due to violations to Nominatim's <a href="https://operations.osmfoundation.org/policies/nominatim/" target="_blank">use policy</a>.

In [1]:
import numpy as np
import geopy

nominatim_geocoder = geopy.geocoders.Nominatim(user_agent = "geocoding_address_blog_post")

After initializing the geocoder, any address can be geocoded simply by passing it as a string to the geocoder. For this exploration we shall use the White House's address at 1600 Pennsylvania Avenue NW.

In [2]:
geocoded_address = nominatim_geocoder.geocode("1600 Pennsylvania Avenue NW")

The geocoded address provides several pieces of information geocoded from the original string. The first piece of information is the full address, again given as a string.

In [3]:
geocoded_address.address

'White House, 1600, Pennsylvania Avenue Northwest, Washington, District of Columbia, 20500, United States'

The second, more useful to machine learning piece of information is the geographical coordinates, given as latitude, longitude and altitude.

In [4]:
geocoded_address.latitude, geocoded_address.longitude, geocoded_address.altitude

(38.897699700000004, -77.03655315, 0.0)

Note that the geographical coordinates are given as decimal degrees instead of degrees minutes seconds. While decimal degrees are arguably more useful in machine learning, degrees minutes seconds are probably more relatable for most people. The function `dd_2_dms` below converts decimal degrees to degrees minutes seconds. 

In [5]:
def dd_2_dms(dd):
    if dd < 0:
        # The algorithm requires the input decimal degrees dd to be positive.
        # We will re-insert any negative signs later on.
        dd = np.abs(dd)
        sign = 0
    else:
        sign = 1
    # dd can be separated as degrees.minutes_seconds at the decimal point by using the modulo at 1.
    degs, mins_secs = np.divmod(dd, 1)
    # minutes_seconds can be further separated as the minutes and seconds by multiplying first min_secs
    # by 60 to get minutes.seconds, and then separating minutes and seconds at the decimal point.
    mins, secs = np.divmod(mins_secs * 60, 1)
    if sign == 0:
        degs = -degs
    # We restrict the precision of seconds to 4 decimal places. This is arbitrary.
    return (degs, mins, np.round(secs * 60, 4))

print(dd_2_dms(geocoded_address.latitude))
print(dd_2_dms(geocoded_address.longitude))

(38.0, 53.0, 51.7189)
(-77.0, 2.0, 11.5913)


In addition to returning the full geocoded address as a string or the geographical coordinates as floating point numbers, geopy also returns a Point object, which is useful for other operations such as performing geometrical intersections with other geometries. 

In [6]:
geocoded_address.point

Point(38.897699700000004, -77.03655315, 0.0)

Finally, using the `raw` attribute of the geocoded address provides the full set of information associate with the geocoded address. This includes the license of the geocoder web service, as well as the type and class of the address involved. For example, the White House is considered to be a government office.

In [7]:
geocoded_address.raw

{'place_id': 159983331,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 238241022,
 'boundingbox': ['38.8974908', '38.897911', '-77.0368537', '-77.0362519'],
 'lat': '38.897699700000004',
 'lon': '-77.03655315',
 'display_name': 'White House, 1600, Pennsylvania Avenue Northwest, Washington, District of Columbia, 20500, United States',
 'class': 'office',
 'type': 'government',
 'importance': 0.9447211541681102}

This ends our exploration in using geopy to geocode string formatted addresses to numerical format using geopy. In addition to geocoding, geopy also offers other geographical functionalities such as geodesic distance calculations. More information can be found on the <a href="https://geopy.readthedocs.io/en/stable/#" target = "_blank">official documentation</a>.